In [13]:
%pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 5.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_validate
from sklearn.metrics import r2_score, accuracy_score, recall_score, roc_auc_score
from xgboost import XGBClassifier, XGBRFClassifier
from lightgbm import LGBMClassifier
import optuna


In [2]:
df_train = pd.read_csv("training.csv")[["Temperatura", "pH", "Aw", "Bw", "Crec"]]
df_validation = pd.read_csv("validation.csv")[["Temperatura", "pH", "Aw", "Bw", "Crec"]]
df = pd.concat([df_train,df_validation])

X_train, X_test, y_train, y_test = train_test_split(
                df.drop("Crec", axis=1),
                df["Crec"],
                train_size = 0.7,
                random_state = 123,
                shuffle = True
            )

cv = RepeatedKFold(n_splits = 5, n_repeats=2, random_state=123)


### XGB training

In [15]:
def objective(trial):
        params = {
                'n_estimators': trial.suggest_int('n_estimators', 1000, 100000, step=10),
                'max_depth': trial.suggest_int('max_depth', 3, 12),
                'scale_pos_weight': trial.suggest_int('scale_pos_weight', 1, 5),
                'learning_rate': trial.suggest_float('learning_rate', 0.00001, 0.01),
                'reg_lambda': trial.suggest_float('reg_lambda', 0.0001, 0.1, log=True),
                'reg_alpha': trial.suggest_float('reg_alpha', 0.0001, 0.1, log=True),
                'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1),
                'subsample': trial.suggest_float('subsample', 0.1, 1),
                }

        model = XGBClassifier(
                n_jobs         = -1,
                random_state   = 123,
                verbosity            = 0,
                enable_categorical = True,
                **params
                )

        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        score = roc_auc_score(y_test, predictions)
        return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5, show_progress_bar=False, timeout=60*10)

print('Mejores hiperparámetros:', study.best_params)
print('Mejor score:', study.best_value)


[I 2025-06-11 09:54:09,203] A new study created in memory with name: no-name-0ceebf45-09f7-4a0c-a6b2-7c6f6b8ca1ea
[W 2025-06-11 09:54:13,846] Trial 0 failed with parameters: {'n_estimators': 59090, 'max_depth': 6, 'scale_pos_weight': 2, 'learning_rate': 0.0069263112939259375, 'reg_lambda': 0.009308930015849952, 'reg_alpha': 0.031641957575913655, 'colsample_bynode': 0.14043331354439387, 'subsample': 0.8585200377363819} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/meta_die/Documents/repositories/datos_Alberto/.venv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_7029/2694636271.py", line 21, in objective
    model.fit(X_train, y_train)
  File "/home/meta_die/Documents/repositories/datos_Alberto/.venv/lib/python3.11/site-packages/xgboost/core.py", line 729, in inner_f
    return func(**kwargs)
           ^^^^^^

KeyboardInterrupt: 

In [11]:
model_best = XGBClassifier(
                n_jobs = -1,
                random_state = 123,
                verbose = 0,
                **study.best_params)

cv_scores = cross_validate(model_best, 
                               X = pd.concat([X_train, X_test]), y = pd.concat([y_train, y_test]), cv = cv,
                               scoring = ("roc_auc", "accuracy"),
                               return_estimator = True, 
                               return_indices = True,
                               return_train_score = True)



/home/meta_die/Documents/repositories/datos_Alberto/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [09:44:13] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/meta_die/Documents/repositories/datos_Alberto/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [09:44:41] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/meta_die/Documents/repositories/datos_Alberto/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [09:45:15] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/meta_die/Documents/repositories/datos_Alberto/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [09:45:54] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "verbose" } are not use

In [17]:
def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 10, 510, step=100),
            'max_depth': trial.suggest_int('max_depth', -1, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.01),
            'subsample': trial.suggest_float('subsample', 0.01, 1),
            'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart']),
            "num_leaves": trial.suggest_int("num_leaves", 1024, 2000)
        }


        model = LGBMClassifier(
                n_jobs         = -1,
                random_state   = 123,
                verbosity            = 0,
                **params
                )

        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        score = roc_auc_score(y_test, predictions)
        return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5, show_progress_bar=False, timeout=60*10)

print('Mejores hiperparámetros:', study.best_params)
print('Mejor score:', study.best_value)


[I 2025-06-11 09:54:40,905] A new study created in memory with name: no-name-4afa6192-599a-4ab1-a943-9bc3d55cb950


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-06-11 09:54:41,309] Trial 0 finished with value: 0.9381917682448023 and parameters: {'n_estimators': 110, 'max_depth': -1, 'learning_rate': 0.0026184286169644447, 'subsample': 0.10454916254113333, 'boosting_type': 'gbdt', 'num_leaves': 1533}. Best is trial 0 with value: 0.9381917682448023.
[I 2025-06-11 09:54:41,328] Trial 1 finished with value: 0.5 and parameters: {'n_estimators': 10, 'max_depth': 5, 'learning_rate': 0.002754608325983556, 'subsample': 0.7773909517357265, 'boosting_type': 'dart', 'num_leaves': 1212}. Best is trial 0 with value: 0.9381917682448023.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-06-11 09:54:41,602] Trial 2 finished with value: 0.5 and parameters: {'n_estimators': 310, 'max_depth': 3, 'learning_rate': 0.00014338383997132575, 'subsample': 0.49279586717424906, 'boosting_type': 'dart', 'num_leaves': 1478}. Best is trial 0 with value: 0.9381917682448023.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-06-11 09:54:41,930] Trial 3 finished with value: 0.6873781364388659 and parameters: {'n_estimators': 310, 'max_depth': 1, 'learning_rate': 0.0064746614674962314, 'subsample': 0.7273148358422874, 'boosting_type': 'dart', 'num_leaves': 1187}. Best is trial 0 with value: 0.9381917682448023.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-06-11 09:54:42,379] Trial 4 finished with value: 0.9323942641071796 and parameters: {'n_estimators': 210, 'max_depth': 7, 'learning_rate': 0.004396595234537054, 'subsample': 0.5656666898192682, 'boosting_type': 'gbdt', 'num_leaves': 1464}. Best is trial 0 with value: 0.9381917682448023.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [19]:
cv_scores

{'fit_time': array([0.45745826, 0.40939665, 0.41691041, 0.43100286, 0.39036155,
        0.39983559, 0.34953117, 0.35554814, 0.36806083, 0.37588239]),
 'score_time': array([0.00444245, 0.00448704, 0.00491858, 0.00419617, 0.00423908,
        0.00420713, 0.00418496, 0.00420809, 0.00710416, 0.00442457]),
 'estimator': [LGBMClassifier(learning_rate=0.0026184286169644447, n_estimators=110, n_jobs=-1,
                 num_leaves=1533, random_state=123, subsample=0.10454916254113333,
                 verbose=0),
  LGBMClassifier(learning_rate=0.0026184286169644447, n_estimators=110, n_jobs=-1,
                 num_leaves=1533, random_state=123, subsample=0.10454916254113333,
                 verbose=0),
  LGBMClassifier(learning_rate=0.0026184286169644447, n_estimators=110, n_jobs=-1,
                 num_leaves=1533, random_state=123, subsample=0.10454916254113333,
                 verbose=0),
  LGBMClassifier(learning_rate=0.0026184286169644447, n_estimators=110, n_jobs=-1,
                 

In [18]:
model_best = LGBMClassifier(
                n_jobs = -1,
                random_state = 123,
                verbose = 0,
                **study.best_params)

cv_scores = cross_validate(model_best, 
                               X = pd.concat([X_train, X_test]), y = pd.concat([y_train, y_test]), cv = cv,
                               scoring = ("roc_auc", "accuracy"),
                               return_estimator = True, 
                               return_indices = True,
                               return_train_score = True)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [23]:
def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 10, 1000, step=10),
            'max_depth': trial.suggest_int('max_depth', 1, 50),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 1e+5, log=True),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 1e+5, log=True),
            'gamma': trial.suggest_float('gamma', 1e-5, 1e+5, log=True),
            'subsample': trial.suggest_float('subsample', 0.1, 1),
            'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1),
            'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.01)
            }


        model = XGBRFClassifier(
                tree_method = 'hist',
                grow_policy = 'depthwise',
                n_jobs         = -1,
                random_state   = 123,
                verbosity            = 0,
                **params
                )

        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        score = roc_auc_score(y_test, predictions)
        return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15, show_progress_bar=False, timeout=60*10)

print('Mejores hiperparámetros:', study.best_params)
print('Mejor score:', study.best_value)

[I 2025-06-11 11:13:26,363] A new study created in memory with name: no-name-ad8e5daa-d33e-459b-a0c9-d67edbc27eec
[I 2025-06-11 11:13:26,550] Trial 0 finished with value: 0.5 and parameters: {'n_estimators': 490, 'max_depth': 31, 'reg_lambda': 0.00839006395759614, 'reg_alpha': 1067.6476942177674, 'gamma': 0.540481699040346, 'subsample': 0.45960466489027807, 'colsample_bynode': 0.20861805461827745, 'learning_rate': 0.008280248614551825}. Best is trial 0 with value: 0.5.
[I 2025-06-11 11:13:26,657] Trial 1 finished with value: 0.5 and parameters: {'n_estimators': 510, 'max_depth': 38, 'reg_lambda': 0.0025729510336180094, 'reg_alpha': 4.310626832363637, 'gamma': 53247.830230892774, 'subsample': 0.7827302978831746, 'colsample_bynode': 0.6925718792155103, 'learning_rate': 0.008589439854720902}. Best is trial 0 with value: 0.5.
[I 2025-06-11 11:13:26,841] Trial 2 finished with value: 0.5 and parameters: {'n_estimators': 510, 'max_depth': 11, 'reg_lambda': 20343.529030344398, 'reg_alpha': 0.0

Mejores hiperparámetros: {'n_estimators': 490, 'max_depth': 31, 'reg_lambda': 0.00839006395759614, 'reg_alpha': 1067.6476942177674, 'gamma': 0.540481699040346, 'subsample': 0.45960466489027807, 'colsample_bynode': 0.20861805461827745, 'learning_rate': 0.008280248614551825}
Mejor score: 0.5
